# <span style="color:#0b486b">  FIT3181: Deep Learning (2022)</span>
***
*CE/Lecturer (Clayton):*  **Dr Trung Le** | trunglm@monash.edu <br/>
*Lecturer (Malaysia):*  **Dr Lim Chern Hong** | lim.chernhong@monash.edu <br/>  <br/>
*Tutor:*  **Mr Thanh Nguyen** \[Thanh.Nguyen4@monash.edu \] |**Mr Tuan Nguyen**  \[tuan.ng@monash.edu \] |**Mr Anh Bui** \[tuananh.bui@monash.edu\] | **Dr Binh Nguyen** \[binh.nguyen1@monash.edu \] | **Mr Md Mohaimenuzzaman** \[md.mohaimen@monash.edu \] |**Mr James Tong** \[james.tong1@monash.edu \]
<br/> <br/>
Faculty of Information Technology, Monash University, Australia
***on Technology, Monash University, Australia
******

## <span style="color:#0b486b">Tutorial 07a: Advanced Convolutional Neural Networks</span> ##

**The purpose of this tutorial is to show you how to implement ResNet, the most popular CNN model. This ResNet implementation follows the original paper of ResNet.**


**Acknowledgement:** *This tutorial is developed based on the material in chapter 7 of the book Dive Into Deep Learning*.

**References and additional reading and resources**
- Chapter 7 of Dive Into Deep Learning ([link](https://d2l.ai/chapter_convolutional-modern/index.html)).

**Hint**: The implementations of ResNet in this tutorial would facilitate you in doing Part 3 of assignment 1. 


## <span style="color:#0b486b">I. Implementation of Residual Network (ResNet)</span> <span style="color:red">****</span> ##
This is a **highly recommend-to-learn** knowledge because ResNet is the most widely popular architecture used in computer vision tasks. Moreover, we can apply what is learned here to assignment 1.  

### <span style="color:#0b486b">I.1. Residual Block</span> ###

The building block of ResNet is the **residual block** which has the following architecture: *[CONV2D, BatchNorm, RELU, CONV2D, BatchNorm]* together with the skip connection. For the skip connection, we have two options: `use 1x1 CONV1D` or `not use 1x1 CONV1D`. The following figure shows the architecture of the residual block for two options.

<img src="./imgs/ResidualBlock.png" align="center" width=600/>

In what follows, we are going to implement the class `Residual`.

In [1]:
import tensorflow as tf

In [2]:
class Residual(tf.keras.Model):
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(num_channels, padding='same', kernel_size=3, strides=strides)
        self.conv2 = tf.keras.layers.Conv2D(num_channels, kernel_size=3, padding='same')
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(num_channels, kernel_size=1, strides=strides)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()

    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3 is not None:
            X = self.conv3(X)
        Y += X
        return tf.keras.activations.relu(Y)

blk = Residual(num_channels=3, use_1x1conv=True, strides=1)
X = tf.random.uniform((10, 32, 32, 3))
Y = blk(X)
print(Y.shape)

(10, 32, 32, 3)


Consider the case when we do not apply 1x1 Conv1D, hence it requires the `num_channels` is equal to the `input_depth` (equal to 3 in the following example).

In [3]:
blk = Residual(3)
X = tf.random.uniform((4, 6, 6, 3))
Y = blk(X)
Y.shape

TensorShape([4, 6, 6, 3])

If we apply 1x1 Conv1D, the output shape of the residual block can be different from the input shape.

In [4]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape

TensorShape([4, 3, 3, 6])

### <span style="color:#0b486b">I.2. ResNet Block</span> ###

We now implement **ResNet block**. A ResNet block consists of several residual blocks and we need to declare the number of residual blocks as `num_residuals` and the common number of channels of residual blocks in a ResNet block as `num_channels`.

In [5]:
class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, num_residuals, first_block=False, **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.residual_layers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual(num_channels))

    def call(self, X):
        for layer in self.residual_layers.layers:
            X = layer(X)
        return X

Note that from the second ResNet block, we apply `1x1 Conv1D` over the skip connection of every first residual block.

### <span style="color:#0b486b">I.3. Residual Network</span> ###

We now declare our ResNet. Note that we set `input_shape = [28,28,1]` because we will train our ResNet on Fashion MNIST dataset.

In [6]:
def create_ResNet():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', input_shape = [28,28,1]),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
        ResnetBlock(64, 2, first_block=True),
        ResnetBlock(128, 2),
        ResnetBlock(256, 2),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(units=10, activation= 'softmax')])

We feed a batch $X$ with batch size $1$ to our ResNet and print out the output shapes. Your task is to explain the results.

In [7]:
X = tf.random.uniform(shape=(1, 28, 28, 1))
for layer in create_ResNet().layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Conv2D output shape:	 (1, 14, 14, 64)
BatchNormalization output shape:	 (1, 14, 14, 64)
Activation output shape:	 (1, 14, 14, 64)
MaxPooling2D output shape:	 (1, 7, 7, 64)
ResnetBlock output shape:	 (1, 7, 7, 64)
ResnetBlock output shape:	 (1, 4, 4, 128)
ResnetBlock output shape:	 (1, 2, 2, 256)
GlobalAveragePooling2D output shape:	 (1, 256)
Dense output shape:	 (1, 10)


### <span style="color:#0b486b">I.4. Test our ResNet</span> ###

In [8]:
import numpy as np

In [9]:
(X_train_full, y_train_full) , (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full, X_test = X_train_full/255.0, X_test/255.0
X_train_full = np.expand_dims(X_train_full, axis=-1)   #expand one more dimension at the end and obtain [60000, 28, 28,1]
X_test = np.expand_dims(X_test, axis=-1)   #expand one more dimension at the end and obtain [10000, 28, 28,1]
X_train, X_valid = X_train_full[:50000,:], X_train_full[50000:,:]
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]
print(X_train_full.shape, y_train_full.shape)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(50000, 28, 28, 1) (50000,)
(10000, 28, 28, 1) (10000,)
(10000, 28, 28, 1) (10000,)


In [10]:
my_resnet = create_ResNet()
my_resnet.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
my_resnet.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
782/782 [==============================] - 76s 96ms/step - loss: 0.4157 - accuracy: 0.8486 - val_loss: 0.4396 - val_accuracy: 0.8420
Epoch 2/5
782/782 [==============================] - 73s 93ms/step - loss: 0.2935 - accuracy: 0.8912 - val_loss: 0.3183 - val_accuracy: 0.8829
Epoch 3/5
782/782 [==============================] - 74s 94ms/step - loss: 0.2557 - accuracy: 0.9041 - val_loss: 0.2847 - val_accuracy: 0.8943
Epoch 4/5
782/782 [==============================] - 74s 94ms/step - loss: 0.2323 - accuracy: 0.9120 - val_loss: 0.2865 - val_accuracy: 0.8952
Epoch 5/5
782/782 [==============================] - 73s 93ms/step - loss: 0.2090 - accuracy: 0.9216 - val_loss: 0.2770 - val_accuracy: 0.8999


**<span style="color:red">Exercise 1</span>**: Implement VGG network and test on Fashion MNIST dataset. Note that you can refer to the code [here](https://d2l.ai/chapter_convolutional-modern/vgg.html).

## <span style="color:#0b486b">II. Additional Reading: New Version of ResNet</span> <span style="color:red">**</span> ## 
This is a **good-to-learn part** and helps you to speculate more recently updated architecture of ResNet.

As additional reading, we are going to implement ResNet which is the state-of-the-art convolutional network for image classification and other tasks which took first place in all three ILSVRC 2015 challenges (classification, detection, and localization). 

ResNet uses **residual blocks** to train Convolutional Neural Networks to depths previously thought impossible. For example, in 2014, the VGG16 and VGG19 architectures were considered very deep. However, with ResNet, we have successfully trained networks with over 100 layers on the challenging ImageNet dataset and over 1,000 layers on CIFAR-10.

What we are going to implement is  the **bottleneck residual block** (proposed by He et al. in their 2016 publication, *Identity Mappings in Deep Residual Networks*) used to train deeper networks which is an extension of the **original residual block** proposed in his publication in 2015.


### <span style="color:#0b486b">II.1. The differences between original and bottleneck residual blocks</span> 

The following figure shows differences between the original residual block and the bottleneck residual block.

<img src='imgs/BottleneckResidualBlock.png' align='center' width=500/>

For the bottleneck one, each block includes:
- ReLU, BATCH-NORM, CONV [filters = K/4, kernel_size = 1x1]
- ReLU, BATCH-NORM, CONV [filters = K/4, kernel_size = 3x3]
- ReLU, BATCH-NORM, CONV [filters = K, kernel_size = 1x1]
- Add inputs and ouputs (residual operation to add shortcut (skip connection) to outputs). This addition is optional for each residual block. In our implementation, when declaring the code for the residual block, we use the parameter `skip_connection` to govern this boolean quantity.

### <span style="color:#0b486b">II.2. Architecture of ResNet </span>

ResNet consists of many consecutive ResNet blocks in which each block has several residual blocks. There are many variants of ResNet architecture, which depends on computer vision tasks. The architecture below is an example that solves image classification on Fashion MNIST dataset. The input data are passed through the following layers:
- Inputs => BATCH-NORM => CONV.
- **ResNet block 1**
  - ***Residual block 1***
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1$, kernel_size = (1,1), strides = (1,1)]
    - Skip connection 
  - ***Residual block 2***
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 
  - .........
  - ***Residual block $N_1$***
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_1$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 
- **ResNet block 2**
  - ***Residual block 1***
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (3,3), strides = (2,2)]  (**downsampling image size by $2$**).
    - ReLU, BATCH-NORM, CONV [filters = $K_2$, kernel_size = (1,1), strides = (1,1)]
    - Skip connection 
  - ***Residual block 2***
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_2$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 
  - .........
  - ***Residual block $N_2$***
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_2/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_2$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 
-  ......................
- **ResNet block M**
  - ***Residual block 1***
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (3,3), strides = (2,2)]  (**downsampling image size by $2$**).
    - ReLU, BATCH-NORM, CONV [filters = $K_M$, kernel_size = (1,1), strides = (1,1)]
    - Skip connection 
  - ***Residual block 2***
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_M$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 
  - .........
  - ***Residual block $N_M$***
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (1,1), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_M/4$, kernel_size = (3,3), strides = (1,1)]
    - ReLU, BATCH-NORM, CONV [filters = $K_M$, kernel_size = (1,1), strides = (1,1)]
    - No skip connection 

### <span style="color:#0b486b">II.3. Implementation of ResNet </span>

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers, regularizers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
from functools import partial
bn_momentum= 0.9
bn_eps= 2E-5
reg= 0.001
DefaultBatchNorm = partial(keras.layers.BatchNormalization, momentum=bn_momentum, epsilon=bn_eps)
DefaultConv2D = partial(keras.layers.Conv2D, kernel_regularizer=regularizers.l2(reg), use_bias=False, padding='same')

In [13]:
class ResNet:
    def __init__(self, num_classes=10, batch_size=32, num_epochs=20, optimizer='adam', learning_rate=0.001,
                 verbose=True, random_state=42):
        super(ResNet, self).__init__()
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.optimizer = keras.optimizers.get(optimizer)
        self.optimizer.learning_rate = learning_rate
        self.random_state = random_state
        self.verbose = verbose
        keras.backend.clear_session()
        np.random.seed(self.random_state)
        tf.random.set_seed(self.random_state)
        self.model = None
    
    @staticmethod
    def ResidualBlock(inputs, K=64, strides= (1,1), skip_connection= False):
        main_layers = [DefaultBatchNorm(), layers.Activation(activation='relu'), DefaultConv2D(filters=int(K/4), kernel_size=1, strides=(1,1)),
                       DefaultBatchNorm(), layers.Activation(activation='relu'), DefaultConv2D(filters=int(K/4), kernel_size=3, strides=strides),
                       DefaultBatchNorm(), layers.Activation(activation='relu'), DefaultConv2D(filters=K, kernel_size=1, strides=(1,1))]
        skip_layers = []
        if skip_connection:
            skip_layers = [DefaultBatchNorm(), layers.Activation(activation='relu'), DefaultConv2D(filters=K, kernel_size=1, strides=strides)]
        
        h = inputs
        for layer in main_layers:
            h = layer(h)
        
        if skip_connection:
            short_cut = inputs
            for layer in skip_layers:
                short_cut = layer(short_cut)
            return (h + short_cut)
        else:
            return h
    
    def build(self, blocks=[3,4], filters=[16,16,16]):
        self.model = models.Model()
        inputs = layers.Input(shape=(28, 28, 1))
        h = inputs
        h = DefaultBatchNorm()(h)
        h = DefaultConv2D(filters=filters[0], kernel_size=3)(h)
        
        for i in range(len(blocks)):
            strides = (1,1) if i==0 else (2,2) # We downsample at every beginning residual block except the first ResNet block
            h = ResNet.ResidualBlock(h, filters[i], strides, True)  # apply the skip connection on the first residual block
            
            for j in range(1, blocks[i]-1, 1): # Add more blocks[i]-1 residual models
                h  =  ResNet.ResidualBlock(h, filters[i], (1,1), False)  # no skip connection for these residual blocks
        
        h = DefaultBatchNorm()(h)
        h = layers.Activation(activation='relu')(h)
        h = layers.AveragePooling2D(pool_size=(8,8))(h)
        h = layers.Flatten()(h)
        h = layers.Dense(units=self.num_classes, activation="softmax")(h)
        self.model = models.Model(inputs=inputs, outputs= h, name="ResNet") # We now have a ResNet model
        self.model.compile(optimizer=self.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X_train, y_train, X_valid=None, y_valid=None, batch_size=None, num_epochs=None, verbose=None):
        batch_size = batch_size if batch_size is not None else self.batch_size
        num_epochs = num_epochs if num_epochs is not None else self.num_epochs
        verbose = verbose if verbose is not None else self.num_epochs
        self.history = self.model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs, verbose=1 if verbose else 0)
    
    def evaluate(self, X_test, y_test):
        loss, acc = self.model.evaluate(X_test, y_test)
        return acc
    
    def plot_progress(self):
        pd.DataFrame(self.history.history).plot(figsize=(8, 5))
        plt.grid(True)
        plt.gca().set_ylim(0, np.max(self.history.history['loss']))  # Set the vertical range to [0-max(train loss)]
        plt.show()
    
    def summary(self):
        print(self.model.summary())
    

In [14]:
res_net = ResNet()

In [15]:
res_net.build()

In [16]:
res_net.summary()

Model: "ResNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 1)   4           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 16)   144         ['batch_normalization[0][0]']    
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 28, 28, 16)  64          ['conv2d[0][0]']            

In [17]:
res_net.fit(X_train, y_train, X_valid, y_valid, batch_size=128, num_epochs=5)

Epoch 1/5
391/391 [==============================] - 67s 167ms/step - loss: 1.7118 - accuracy: 0.4035 - val_loss: 1.3330 - val_accuracy: 0.5590
Epoch 2/5
391/391 [==============================] - 66s 168ms/step - loss: 1.0402 - accuracy: 0.6899 - val_loss: 0.9108 - val_accuracy: 0.7335
Epoch 3/5
391/391 [==============================] - 66s 168ms/step - loss: 0.8514 - accuracy: 0.7388 - val_loss: 0.8227 - val_accuracy: 0.7460
Epoch 4/5
391/391 [==============================] - 66s 168ms/step - loss: 0.7690 - accuracy: 0.7570 - val_loss: 0.7471 - val_accuracy: 0.7548
Epoch 5/5
391/391 [==============================] - 66s 169ms/step - loss: 0.7271 - accuracy: 0.7672 - val_loss: 0.7178 - val_accuracy: 0.7758


In [18]:
res_net.evaluate(X_test, y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.7231 - accuracy: 0.7750


0.7749999761581421

In [19]:
res_net.plot_progress()

---
### <span style="color:#0b486b"> <div  style="text-align:center">**THE END**</div> </span>